In [1]:
import pandas as pd
import os
from collections import defaultdict
import imp
import sys
import re
import time
import json
import pickle
from tqdm import tqdm
import numpy as np
import utils
from multiprocessing import Pool
import pdb
from sklearn.model_selection import train_test_split

In [2]:
sys.path.append('/home/zilong.zhang1/AE_automation/ClarityNLP/nlp')
sys.path.append('/home/zilong.zhang1/AE_automation/ClarityNLP/')
import Sentence_tokenizer_from_Clarity as cla_token

# Create a dictionary that map HADM_IDs to ICD codes
icd_lines =  open('../data/DIAGNOSES_ICD.csv').readlines()
adm_icd_dict = defaultdict(lambda:set())
for i in range(len(icd_lines)):
    split_line = icd_lines[i].split(',')
    adm = split_line[2]
    adm_icd_dict[adm].add(split_line[-1].strip()[1:-1])
#json.dump(adm_icd_dict,open('./result/adm_icd_dict.json','w')) # Write this dictionary to file so we don't need to do it agian


No section: 'ohdsi'
Initializing models for term finder...
section_tagger_init...
Context init...
Context init...
Segmentation init...
Done initializing models for term finder...
Initializing models for value extractor...
Done initializing models for value extractor...
Initializing models for measurement finder...
Done initializing models for measurement finder..


In [3]:
adm_icdlist_dict = {}
for k,v in adm_icd_dict.items():
    adm_icdlist_dict[k] = list(v)
json.dump(adm_icdlist_dict,open('./result/adm_icdlist_dict.json','w'))

In [4]:

# Find all HADM_IDs that have only one discharge summary described as a report.
# And create a dictionary that map those HADM_IDs to their DSs
note_df = pd.read_csv('../data/NOTEEVENTS.csv')
adm_count_dict = {}
ds_note = note_df.loc[note_df['CATEGORY'] == 'Discharge summary']
valid_ds_note = ds_note.loc[ds_note['DESCRIPTION'] == 'Report']
add_ds_note = ds_note.loc[ds_note['DESCRIPTION'] != 'Report']
raw_hadm_list =  list(valid_ds_note['HADM_ID'])
for hadm in raw_hadm_list: # Count how many valid DSs one HADM_ID has
    key = hadm
    value = raw_hadm_list.count(hadm)
    adm_count_dict[key] = value
    
valid_hadm_id = [] # The list of 
for key,value in adm_count_dict.items():
    if value == 1:
        valid_hadm_id.append(key)
pickle.dump(valid_hadm_id,open('./result/valid_hadm_id_list','wb')) #Save to disc


/home/zilong.zhang1/miniconda3/envs/py35/lib/python3.5/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
adm_text_dict = {}
for index,item in valid_ds_note.iterrows():
    if item['HADM_ID'] in valid_hadm_id:
        if item['HADM_ID'] in adm_text_dict:
            pdb.set_trace()
        adm_text_dict[item['HADM_ID']] = item['TEXT']
        
for index,item in add_ds_note.iterrows(): # merge addendum and report
    if item['HADM_ID'] in valid_hadm_id:
        adm_text_dict[item['HADM_ID']] = adm_text_dict[item['HADM_ID']] + str(item['TEXT'])

json.dump(adm_text_dict,open('./result/adm_text_dict.json','w'))



In [6]:
valid_num = len(valid_hadm_id)
#from tqdm import tqdm
print(valid_num)
def _cla_seg(start):
    begin_time = time.time()
    tmp_adm_seg_dict = {}
    #list_of_list_sent = []
    for i in range(start, min(valid_num,start + num)):
        text = adm_text_dict[valid_hadm_id[i]]
        text = re.sub(r'\s*\[.*?\]\s*',' ',text)
        list_sent = cla_token.parse_sentences_spacy(text)
        tmp_adm_seg_dict[valid_hadm_id[i]] = list_sent
        #list_of_list_sent.append(list_sent)
    print(start)
    print(time.time() - begin_time)
    return tmp_adm_seg_dict


50563


In [7]:
1395 + 10116

11511

In [10]:
import regex as re
re.__version__

'2.4.139'

In [11]:
re

<module 'regex' from '/home/zilong.zhang1/miniconda3/envs/py35/lib/python3.5/site-packages/regex.py'>

In [12]:
i = 11511
text = adm_text_dict[valid_hadm_id[i]]
text = re.sub(r'\s*\[.*?\]\s*',' ',text)
list_sent = cla_token.parse_sentences_spacy(text)

Segmentation init...


In [13]:
list_sent

['Admission Date:',
 'Discharge Date: Date of Birth:',
 'Sex: F Service:',
 'MEDICINE Allergies:',
 'Amoxicillin / lisinopril',
 'Attending:',
 'Chief Complaint: Shortness of breath Major Surgical or Invasive Procedure:',
 'Central venous line placement and removal History of Present Illness:',
 'Pt is a 62 yo F with emphysema and DMII who presents with fevers and shortness of breath.',
 'Pt reports runny nose for the past week, in addition to cough non-productive of sputum.',
 'Two days before admission, she experienced chills and rigors.',
 'She also report today a fever up to 105.1 at home, some muscle aches in her arms.',
 'Denies CP but admits to some SOB and DOE.',
 'Took some Tylenol efor the fever and took some vicodin for a headache on Saturday (which is currently not present).',
 'Experienced some nausea on the way to the ED.',
 'Denies neck stiffness or photophobia.',
 'Her symptoms worsened to a point where her daughters urged her to come for evaluation today.',
 "Reports s

In [23]:
%%time
from tqdm import tqdm
num = int(np.ceil(valid_num/thread_num))
tmp_adm_seg_dict = {}
#list_of_list_sent = []
for i in tqdm(range(10116,10116+num)):
    text = adm_text_dict[valid_hadm_id[i]]
    text = re.sub(r'\s*\[.*?\]\s*',' ',text)
    list_sent = cla_token.parse_sentences_spacy(text)
    tmp_adm_seg_dict[valid_hadm_id[i]] = list_sent
    #list_of_list_sent.append(list_sent)


 55%|█████▌    | 1395/2529 [36:33<29:43,  1.57s/it]


KeyboardInterrupt: 

In [27]:
len(adm_text_dict[valid_hadm_id[i]])

12870

In [31]:
text

"Admission Date: Discharge Date: Date of Birth: Sex:   F\n\nService: MEDICINE\n\nAllergies:\nAmoxicillin / lisinopril\n\nAttending: Chief Complaint:\nShortness of breath\n\nMajor Surgical or Invasive Procedure:\nCentral venous line placement and removal\n\n\nHistory of Present Illness:\nPt is a 62 yo F with emphysema and DMII who presents with fevers\nand shortness of breath. Pt reports runny nose for the past\nweek, in addition to cough non-productive of sputum. Two days\nbefore admission, she experienced chills and rigors. She also\nreport today a fever up to 105.1 at home, some muscle aches in\nher arms. Denies CP but admits to some SOB and DOE. Took some\nTylenol efor the fever and took some vicodin for a headache on\nSaturday (which is currently not present). Experienced some\nnausea on the way to the ED. Denies neck stiffness or\nphotophobia. Her symptoms worsened to a point where her\ndaughters urged her to come for evaluation today. Reports some\n'black' diarrhea in the ED, non

In [33]:
adm_text_dict[valid_hadm_id[i]]

"Admission Date:  [**2169-1-28**]              Discharge Date:   [**2169-2-2**]\n\nDate of Birth:  [**2107-1-5**]             Sex:   F\n\nService: MEDICINE\n\nAllergies:\nAmoxicillin / lisinopril\n\nAttending:[**First Name3 (LF) 348**]\nChief Complaint:\nShortness of breath\n\nMajor Surgical or Invasive Procedure:\nCentral venous line placement and removal\n\n\nHistory of Present Illness:\nPt is a 62 yo F with emphysema and DMII who presents with fevers\nand shortness of breath. Pt reports runny nose for the past\nweek, in addition to cough non-productive of sputum. Two days\nbefore admission, she experienced chills and rigors. She also\nreport today a fever up to 105.1 at home, some muscle aches in\nher arms. Denies CP but admits to some SOB and DOE. Took some\nTylenol efor the fever and took some vicodin for a headache on\nSaturday (which is currently not present). Experienced some\nnausea on the way to the ED. Denies neck stiffness or\nphotophobia. Her symptoms worsened to a point w

In [14]:
thread_num = 30
num = int(np.ceil(valid_num/thread_num))
p = Pool(thread_num)
adm_seg_dict_list = p.map(_cla_seg,range(0,valid_num,num))


10116
998.0323791503906
38778
1022.0925068855286
16860
1023.0536890029907
32034
1035.328101158142
20232
1035.500895500183
48894
1036.4719183444977
21918
1043.7946379184723
37092
1044.571340084076
45522
1044.9047312736511
5058
1051.4785523414612
40464
1054.7393729686737
26976
1054.9119791984558
43836
1055.6633729934692
42150
1057.6639807224274
30348
1060.7483863830566
35406
1060.8630797863007
18546
1066.0315034389496
6744
1066.0633397102356
33720
1070.2176361083984
13488
1070.3424143791199
0
1071.5607867240906
28662
1075.0414061546326
8430
1076.7518191337585
25290
1078.854026556015
47208
1081.6601407527924
1686
1091.2015342712402
11802
1098.0746047496796
23604
1100.151210308075
15174
1113.8751890659332
3372
1123.3857743740082


In [16]:
list(range(0,valid_num,num))

[0,
 2529,
 5058,
 7587,
 10116,
 12645,
 15174,
 17703,
 20232,
 22761,
 25290,
 27819,
 30348,
 32877,
 35406,
 37935,
 40464,
 42993,
 45522,
 48051]

In [ ]:
37935,48051,12645,7587,35406,2529,25290,27819,0,15174,20232,5058,42993,
0, 2529, 5058, 7587, 10116, 12645, 15174, 17703, 20232, 22761, 25290, 27819, 30348, 32877, 35406, 37935, 40464, 42993, 45522, 48051

In [19]:
set_a = set([37935,48051,12645,7587,35406,2529,25290,27819,0,15174,20232,5058,42993])
set_b = set([0, 2529, 5058, 7587, 10116, 12645, 15174, 17703, 20232, 22761, 25290, 27819, 30348, 32877, 35406, 37935, 40464, 42993, 45522, 48051])

In [21]:
set_b.difference(set_a)

{10116, 17703, 22761, 30348, 32877, 40464, 45522}

In [17]:
48051 + num

50580

In [15]:
2529 * 20

50580

In [15]:
adm_seg_dict = {}
for i in range(len(adm_seg_dict_list)):
    adm_seg_dict.update(adm_seg_dict_list[i])
print(len(adm_seg_dict.keys()))
json.dump(adm_seg_dict,open('./adm_seg_dict.json','w'))


50563


# Run start from here

In [16]:
AE_icds = ['99859','9093','99667','99666']

In [18]:
adm_with_AE = []
prev_key = 0
for key,value in adm_icd_dict.items():
    for icd in AE_icds:
        if icd in value:
            adm_with_AE.append(key)
            break
len(adm_with_AE)
pickle.dump(adm_with_AE,open('./valid_AE_adm','wb'))

AE_adm_seg_dict = {}
Neg_adm_seg_dict = {}

for key,value in adm_seg_dict.items():
    if str(key)[:-2] in adm_with_AE:
        AE_adm_seg_dict[key] = value
    else:
        Neg_adm_seg_dict[key] = value
print(len(AE_adm_seg_dict))
print(len(Neg_adm_seg_dict))

json.dump(AE_adm_seg_dict,open('./result/AE_adm_seg_dict','w'))
json.dump(Neg_adm_seg_dict,open('./result/Neg_adm_seg_dict','w'))

adm_list = []
total_seg_list = []
label_list = []
ratio = 2
count = 0
neg_num = len(AE_adm_seg_dict)*ratio
for key,value in AE_adm_seg_dict.items():
    adm_list.append(key)
    label_list.append(1)
    total_seg_list.append(value)

for key,value in Neg_adm_seg_dict.items():
    adm_list.append(key)
    label_list.append(0)
    total_seg_list.append(value)
    count +=1
    if count == neg_num:
        break
        


1203
49360


In [22]:
X_train, X_test, y_train, y_test,adm_train,adm_test = train_test_split(total_seg_list,label_list,adm_list,stratify=label_list,test_size=0.2)

data_dir = './result/ratio2/'
with open(data_dir + 'text_train.pkl', 'wb') as f:
    pickle.dump(X_train, f)
with open(data_dir + 'text_test.pkl', 'wb') as f:
    pickle.dump(X_test, f)
with open(data_dir + 'label_train.pkl', 'wb') as f:
    pickle.dump(y_train, f)
with open(data_dir + 'label_test.pkl', 'wb') as f:
    pickle.dump(y_test, f)
with open(data_dir + 'hadm_train.pkl', 'wb') as f:
    pickle.dump(adm_train, f)
with open(data_dir + 'hadm_test.pkl', 'wb') as f:
    pickle.dump(adm_test, f)


In [20]:
len(adm_with_AE)


1307